# Baseline Normalized Data Notebook

This notebook is collection of all the stuff that we tried during researching and working with the baseline normalized data.
<br> <b>Disclamer: Feature selection and hyperparamether optimization is not present in this notebook. Plus these stuff aren't helping too much to have a better results in the end.

In [1]:
# Module for Auto Visualization
!pip install AutoViz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 499.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 6.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 25.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.1 MB/s eta 0:00:00
  Attempting uninstall: panel
    Found existing installation: panel 0.13.1
    Uninstalling panel-0.13.1:
      Successfully uninstalled panel-0.13.1


In [107]:
# Loading all the modules that I will use in this notebook
from itertools import product

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from autoviz.AutoViz_Class import AutoViz_Class

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

import cudf, cuml

import lightgbm as lgb

from xgboost import XGBRegressor
from cuml.neighbors import KNeighborsRegressor as knnCMR
from cuml.ensemble import RandomForestRegressor as rfCMR

from xgboost import XGBClassifier
from cuml.neighbors import KNeighborsClassifier as knnCMC
from cuml.ensemble import RandomForestClassifier as rfCMC

from cuml.svm import SVC
from cuml.linear_model import LogisticRegression
from cuml.multiclass import OneVsRestClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingRegressor

## 1. Load Data

In [3]:
# Loading the train, test and validation data(datasets) + looking the first 5 rows of the train dataset as confirmation that the data is loaded
train = pd.read_csv('../input/classification-baseline-normalization-emg/classification/train/videos_emg_features_05_01_train.csv')
validation = pd.read_csv('../input/classification-baseline-normalization-emg/classification/validation/videos_emg_features_05_01_validation.csv')
test = pd.read_csv('../input/classification-baseline-normalization-emg/classification/test/videos_emg_features_05_01_test.csv')
train.head()

,Subject,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,...,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx,video,arousal,valence,class
0,Subject 26,0.000178,3.184661e-07,-0.043520,0.161232,-0.212268,0.228554,0.043520,-0.185033,0.104441,...,1.342110e-15,3.368695e-13,-86218.958346,-86218.958346,-0.014856,1.377585e-07,GroupA\01,4,5,neutral
1,Subject 26,0.000162,2.598283e-07,0.024016,0.083687,-0.115319,0.209899,-0.024016,-0.092173,0.071753,...,1.118638e-15,2.807781e-13,-83722.723328,-83722.723328,-0.018930,1.184581e-07,GroupA\01,4,5,neutral
2,Subject 26,0.000130,1.856267e-07,0.039356,0.046684,-0.054038,0.186081,-0.039356,-0.050375,0.035060,...,1.150659e-15,2.888153e-13,-66501.839605,-66501.839605,-0.007971,1.015204e-07,GroupA\01,4,5,neutral
3,Subject 26,0.000129,1.931239e-07,0.056126,0.036243,-0.053642,0.165240,-0.056126,-0.038258,0.039529,...,1.033205e-15,2.593346e-13,-66515.135789,-66515.135789,-0.009963,8.059755e-08,GroupA\01,4,5,neutral
4,Subject 26,0.000151,2.156509e-07,0.048009,0.063722,-0.051171,0.181816,-0.048009,-0.068797,0.025145,...,1.064508e-15,2.671916e-13,-75919.081446,-75919.081446,-0.015845,7.440904e-08,GroupA\01,4,5,neutral


## 2. Basic Data Exploration

In [4]:
# Basic info about the size(rows+colums) of the dataset, the memory usage and the datatypes of the data in the train dataset
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142596 entries, 0 to 142595
Columns: 243 entries, Subject to class
dtypes: float64(238), int64(2), object(3)
memory usage: 264.4+ MB


In [5]:
# Basic info about the size(rows+colums) of the dataset, the memory usage and the datatypes of the data in the validation dataset
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34022 entries, 0 to 34021
Columns: 243 entries, Subject to class
dtypes: float64(238), int64(2), object(3)
memory usage: 63.1+ MB


In [6]:
# Basic info about the size(rows+colums) of the dataset, the memory usage and the datatypes of the data in the test dataset
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30791 entries, 0 to 30790
Columns: 243 entries, Subject to class
dtypes: float64(238), int64(2), object(3)
memory usage: 57.1+ MB


In [7]:
# The names of the columns(features) of the train dataset(basically they are same for the train, validation and test datasets)
print(train.columns.to_list())

['Subject', '0_iav', '0_aac', '0_ar1', '0_ar2', '0_ar3', '0_ar4', '0_cc1', '0_cc2', '0_cc3', '0_cc4', '0_dasdv', '0_kurt', '0_log', '0_mav1', '0_mav2', '0_mav', '0_mhw', '0_mtw', '0_rms', '0_skew', '0_ssi', '0_tm', '0_var', '0_v_order', '0_wl', '0_mnf', '0_mdf', '0_pkf', '0_mnp', '0_ttp', '0_vcf', '0_psr', '0_ohm', '0_maxx', '1_iav', '1_aac', '1_ar1', '1_ar2', '1_ar3', '1_ar4', '1_cc1', '1_cc2', '1_cc3', '1_cc4', '1_dasdv', '1_kurt', '1_log', '1_mav1', '1_mav2', '1_mav', '1_mhw', '1_mtw', '1_rms', '1_skew', '1_ssi', '1_tm', '1_var', '1_v_order', '1_wl', '1_mnf', '1_mdf', '1_pkf', '1_mnp', '1_ttp', '1_vcf', '1_psr', '1_ohm', '1_maxx', '2_iav', '2_aac', '2_ar1', '2_ar2', '2_ar3', '2_ar4', '2_cc1', '2_cc2', '2_cc3', '2_cc4', '2_dasdv', '2_kurt', '2_log', '2_mav1', '2_mav2', '2_mav', '2_mhw', '2_mtw', '2_rms', '2_skew', '2_ssi', '2_tm', '2_var', '2_v_order', '2_wl', '2_mnf', '2_mdf', '2_pkf', '2_mnp', '2_ttp', '2_vcf', '2_psr', '2_ohm', '2_maxx', '3_iav', '3_aac', '3_ar1', '3_ar2', '3_ar3'

In [8]:
# The datatypes of each feature
print(train.dtypes.to_list())

[dtype('O'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'

In [9]:
# All the features which datatype is object(string)
print(train.select_dtypes(include=['object']).columns.to_list())

['Subject', 'video', 'class']


In [10]:
# The shape of the train, validation, test datasets(number of rows + colums)
print(train.shape)
print(validation.shape)
print(test.shape)

(142596, 243)
(34022, 243)
(30791, 243)


In [11]:
# Looking if there are any NaN/Null values in the train dataset
train.isna().sum().sum()

0

In [12]:
# Looking if there are any NaN/Null values in the validation dataset
validation.isna().sum().sum()

0

In [13]:
# Looking if there are any NaN/Null values in the test dataset
test.isna().sum().sum()

0

## 3. EDA, Visualization

In [14]:
# Getting the unique values of the valence column from the train dataset
valences=train["valence"]
unique_valences, num_examples_per_valence = np.unique(valences, return_counts = True)
unique_valences

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
# Getting the unique values of the class column from the train dataset
classes=train["class"]
unique_classes, num_examples_per_class = np.unique(classes, return_counts = True)
unique_classes

array(['negative', 'neutral', 'positive'], dtype=object)

In [17]:
# Calculating the percentages of how many entries are negative, neutral and positive(from the class)
negative_perc=num_examples_per_class[0]/(num_examples_per_class.sum())*100
positive_perc=num_examples_per_class[2]/(num_examples_per_class.sum())*100
neutral_perc=num_examples_per_class[1]/(num_examples_per_class.sum())*100
print(neutral_perc)
print(positive_perc)
print(negative_perc)

51.89907150270695
21.6990658924514
26.401862604841654


In [21]:
# Plotting the distribution of the valence per entries
fig, ax = plt.subplots(nrows=1, ncols=1, figsize = (15, 15))

# define the color palette which we will use
cmap = plt.get_cmap("tab20")
# sample as many colors from the palette as there are unique activites, so that each activity will be represented by a different color
sampled_colors = [cmap(i) for i in range (len(num_examples_per_valence))]

# use the pie chart function in the pyplot module to visualize the activity distributions
ax.pie(x = num_examples_per_valence, 
       colors = sampled_colors,
       labels = unique_valences, 
       wedgeprops= {'linewidth': 1, 
                    'edgecolor': 'black'}, 
       textprops = {'size': 'large', 
                    'family': "monospace",
                    'weight': 'medium'} 
);

In [22]:
# Plotting the distribution of classes per entry
fig, ax = plt.subplots(nrows=1, ncols=1, figsize = (15, 15))
# define the color palette which we will use
cmap = plt.get_cmap("tab20")
neg= "{:.2f}".format(negative_perc)
neut= "{:.2f}".format(neutral_perc)
pos = "{:.2f}".format(positive_perc)
my_labels=['','','']
my_labels[0]=unique_classes[0]+" "+str(neg)+"%"
my_labels[1]=unique_classes[1]+" "+str(neut)+"%"
my_labels[2]=unique_classes[2]+" "+str(pos)+"%"
# sample as many colors from the palette as there are unique activites, so that each activity will be represented by a different color
sampled_colors = [cmap(i) for i in range (len(num_examples_per_class))]
ax.pie(x = num_examples_per_class, 
       colors = sampled_colors,
       labels = my_labels, 
       wedgeprops= {'linewidth': 1, 
                    'edgecolor': 'black'}, 
       textprops = {'size': 'large', 
                    'family': "monospace",
                    'weight': 'medium'} 
);

In [23]:
AV = AutoViz_Class()
viz = AV.AutoViz('../input/classification-baseline-normalization-emg/classification/train/videos_emg_features_05_01_train.csv')

Shape of your Data Set loaded: (142596, 243)
Not able to read or load file. Please check your inputs and try again...


## 4. Data Preprocessing + Data Split

In [24]:
# From the train dataset dropping all the columns(info) about the subject, video, arousal, valence
train = train.drop(['Subject' ,'video', 'arousal'], axis = 1)
train.head()

,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,0_cc4,...,6_mdf,6_pkf,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx,valence,class
0,0.000178,3.184661e-07,-0.043520,0.161232,-0.212268,0.228554,0.043520,-0.185033,0.104441,-0.433120,...,-137.849854,-38.453445,1.342110e-15,3.368695e-13,-86218.958346,-86218.958346,-0.014856,1.377585e-07,5,neutral
1,0.000162,2.598283e-07,0.024016,0.083687,-0.115319,0.209899,-0.024016,-0.092173,0.071753,-0.374520,...,-107.129854,53.706555,1.118638e-15,2.807781e-13,-83722.723328,-83722.723328,-0.018930,1.184581e-07,5,neutral
2,0.000130,1.856267e-07,0.039356,0.046684,-0.054038,0.186081,-0.039356,-0.050375,0.035060,-0.306471,...,-148.089854,53.706555,1.150659e-15,2.888153e-13,-66501.839605,-66501.839605,-0.007971,1.015204e-07,5,neutral
3,0.000129,1.931239e-07,0.056126,0.036243,-0.053642,0.165240,-0.056126,-0.038258,0.039529,-0.277746,...,-86.649854,53.706555,1.033205e-15,2.593346e-13,-66515.135789,-66515.135789,-0.009963,8.059755e-08,5,neutral
4,0.000151,2.156509e-07,0.048009,0.063722,-0.051171,0.181816,-0.048009,-0.068797,0.025145,-0.291858,...,-76.409854,43.466555,1.064508e-15,2.671916e-13,-75919.081446,-75919.081446,-0.015845,7.440904e-08,5,neutral


In [25]:
# From train dataset remove every second row with neutral class to basically undersample it
train_no_neutral = train[train["class"] != "neutral"]
train_neutral = train[train["class"] == "neutral"]
train_neutral = train_neutral.iloc[::2,:]
train_under = pd.concat([train_no_neutral, train_neutral])

In [26]:
# Seeing if the train undersampled dataset is okay
train_under

,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,0_cc4,...,6_mdf,6_pkf,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx,valence,class
1425,-0.000059,-1.036992e-07,-0.054061,0.001648,-0.091893,-0.069147,0.054061,-0.002167,0.090674,0.064159,...,-45.689854,84.426555,-1.642886e-16,-4.123644e-14,-5044.325552,-5044.325552,0.004006,-1.450892e-08,8,positive
1426,-0.000071,-1.349660e-07,-0.031998,0.023360,-0.075817,-0.041584,0.031998,-0.026572,0.064956,0.031417,...,-55.929854,84.426555,-1.134881e-16,-2.848552e-14,-15532.008879,-15532.008879,0.002330,2.640986e-09,8,positive
1427,-0.000060,-1.205187e-07,-0.041297,0.070463,-0.072981,-0.009469,0.041297,-0.080807,0.039630,-0.006469,...,-35.449854,-69.173445,-1.506899e-16,-3.782317e-14,-4398.964884,-4398.964884,0.003444,9.167321e-09,8,positive
1428,-0.000059,-1.144399e-07,-0.035109,0.043961,-0.058351,-0.007829,0.035109,-0.050202,0.039067,-0.008720,...,-14.969854,-69.173445,-1.911886e-16,-4.798835e-14,612.948582,612.948582,0.002385,1.045101e-09,8,positive
1429,-0.000066,-1.108408e-07,-0.035689,0.051631,-0.070123,-0.010355,0.035689,-0.058991,0.046165,-0.008493,...,36.230146,2.506555,-2.842338e-16,-7.134267e-14,16220.881162,16220.881162,0.001083,-6.076149e-09,8,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139718,0.000256,3.292045e-07,-0.047147,0.092537,-0.035995,0.383292,0.047147,-0.105354,-0.003605,-0.605533,...,-131.604517,-32.114244,5.144911e-16,1.291373e-13,-55344.039619,-55344.039619,0.010749,8.843681e-08,6,neutral
139720,0.000212,2.836518e-07,0.007930,0.071219,-0.061305,0.384536,-0.007930,-0.080105,0.031233,-0.637101,...,-131.604517,-83.314244,1.970647e-16,4.946323e-14,-41377.312753,-41377.312753,0.016530,3.205913e-08,6,neutral
139722,0.000197,2.599762e-07,0.007858,0.068786,-0.076338,0.385370,-0.007858,-0.077362,0.047100,-0.651964,...,-90.644517,131.725756,-6.430514e-17,-1.614059e-14,-27133.350851,-27133.350851,0.013757,1.827166e-08,6,neutral
139724,0.000174,2.101631e-07,0.000765,0.109968,-0.160012,0.406859,-0.000765,-0.124028,0.104304,-0.737949,...,-59.924517,101.005756,-1.239090e-17,-3.110115e-15,-9810.217996,-9810.217996,0.005868,-5.877859e-09,6,neutral


In [27]:
# Getting the unique values of the class column from the train dataset
classes=train_under["class"]
unique_classes, num_examples_per_class = np.unique(classes, return_counts = True)
unique_classes

array(['negative', 'neutral', 'positive'], dtype=object)

In [28]:
# The neutral class in percentage
neutral_perc=num_examples_per_class[1]/(num_examples_per_class.sum())*100
neutral_perc

35.04304262593164

In [29]:
# The shape of the train(undersampled) data
train_under.shape

(105593, 240)

In [30]:
# From the validation dataset dropping all the columns(info) about the subject, video, arousal, valence
validation = validation.drop(['Subject' ,'video', 'arousal'], axis = 1)
validation.head()

,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,0_cc4,...,6_mdf,6_pkf,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx,valence,class
0,-0.000169,-4.098298e-07,-0.060356,0.047298,0.147317,-0.008989,0.060356,-0.054022,-0.169171,0.064959,...,-54.347199,-309.075322,-8.735187e-17,-2.192532e-14,10452.289503,10452.289503,0.011846,-9.987874e-09,8,positive
1,-0.000141,-3.802300e-07,-0.070874,0.051835,0.222165,0.040910,0.070874,-0.059434,-0.243970,0.018670,...,-218.187199,-309.075322,9.984347e-18,2.506071e-15,27763.523048,27763.523048,0.036112,3.352146e-10,8,positive
2,-0.000144,-3.757562e-07,-0.084546,0.071872,0.215971,0.057898,0.084546,-0.083216,-0.244595,-0.004854,...,-177.227199,-309.075322,1.522556e-17,3.821615e-15,44008.348898,44008.348898,0.035138,9.668015e-09,8,positive
3,-0.000169,-4.161795e-07,0.006791,0.079512,0.134118,0.029853,-0.006791,-0.089954,-0.169243,0.006249,...,-44.107199,-298.835322,-6.914059e-17,-1.735429e-14,44420.768513,44420.768513,0.020813,-6.116931e-09,8,positive
4,-0.000182,-4.401250e-07,0.024188,0.062458,0.158687,0.047232,-0.024188,-0.070336,-0.185811,-0.013414,...,-33.867199,438.444678,-6.811706e-17,-1.709738e-14,27132.481327,27132.481327,0.015002,7.444352e-10,8,positive


In [31]:
# From the test dataset dropping all the columns(info) about the subject, video, arousal, valence
test = test.drop(['Subject' ,'video', 'arousal'], axis = 1)
test.head()

,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,0_cc4,...,6_mdf,6_pkf,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx,valence,class
0,-0.000037,3.367405e-08,0.001367,-0.086367,-0.116219,-0.226394,-0.001367,0.099223,0.162996,0.303384,...,147.831579,15.351579,-1.008898e-16,-2.532334e-14,6392.686888,6392.686888,-0.022298,-9.077232e-09,4,neutral
1,-0.000049,-1.305943e-08,0.020567,-0.091839,-0.112770,-0.198270,-0.020567,0.104707,0.161792,0.254321,...,168.311579,15.351579,-1.258649e-16,-3.159208e-14,18021.302083,18021.302083,-0.022005,-5.601545e-09,4,neutral
2,-0.000085,-7.680120e-08,0.039382,-0.074769,-0.087634,-0.220493,-0.039382,0.084808,0.125708,0.302672,...,158.071579,15.351579,-1.343167e-16,-3.371349e-14,-1806.291597,-1806.291597,-0.027671,-1.806795e-08,4,neutral
3,-0.000088,-1.064329e-07,0.000229,-0.021488,-0.100027,-0.168621,-0.000229,0.025195,0.112178,0.218988,...,199.031579,25.591579,-1.217489e-16,-3.055896e-14,-6582.861480,-6582.861480,-0.034137,-1.273636e-08,4,neutral
4,-0.000097,-1.259941e-07,0.029450,-0.046020,-0.079286,-0.161504,-0.029450,0.052715,0.103023,0.210206,...,168.311579,25.591579,-1.561588e-16,-3.919586e-14,-2012.627914,-2012.627914,-0.030596,-1.440579e-08,4,neutral


In [59]:
# The train data is all the columns except the last column which is the class
X_train = train.iloc[:,:-2]
X_train

,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,0_cc4,...,6_wl,6_mnf,6_mdf,6_pkf,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx
0,0.000178,3.184661e-07,-0.043520,0.161232,-0.212268,0.228554,0.043520,-0.185033,0.104441,-0.433120,...,-4.691394e-06,-121.613493,-137.849854,-38.453445,1.342110e-15,3.368695e-13,-86218.958346,-86218.958346,-0.014856,1.377585e-07
1,0.000162,2.598283e-07,0.024016,0.083687,-0.115319,0.209899,-0.024016,-0.092173,0.071753,-0.374520,...,-2.198235e-06,-103.376845,-107.129854,53.706555,1.118638e-15,2.807781e-13,-83722.723328,-83722.723328,-0.018930,1.184581e-07
2,0.000130,1.856267e-07,0.039356,0.046684,-0.054038,0.186081,-0.039356,-0.050375,0.035060,-0.306471,...,4.908416e-06,-100.451320,-148.089854,53.706555,1.150659e-15,2.888153e-13,-66501.839605,-66501.839605,-0.007971,1.015204e-07
3,0.000129,1.931239e-07,0.056126,0.036243,-0.053642,0.165240,-0.056126,-0.038258,0.039529,-0.277746,...,7.360909e-07,-94.487335,-86.649854,53.706555,1.033205e-15,2.593346e-13,-66515.135789,-66515.135789,-0.009963,8.059755e-08
4,0.000151,2.156509e-07,0.048009,0.063722,-0.051171,0.181816,-0.048009,-0.068797,0.025145,-0.291858,...,5.374088e-07,-96.068424,-76.409854,43.466555,1.064508e-15,2.671916e-13,-75919.081446,-75919.081446,-0.015845,7.440904e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142591,0.000094,1.112939e-07,-0.098656,0.063347,0.029774,0.171195,0.098656,-0.072003,-0.054811,-0.244647,...,-1.968631e-07,44.360198,114.155483,-32.114244,3.072567e-16,7.712143e-14,-35193.391144,-35193.391144,-0.029496,2.397397e-08
142592,0.000106,9.953773e-08,-0.084092,0.062991,0.020209,0.219874,0.084092,-0.071479,-0.045229,-0.323285,...,-1.810208e-06,30.668089,62.955483,60.045756,2.760531e-16,6.928934e-14,-19437.455821,-19437.455821,-0.018619,3.858723e-08
142593,0.000111,1.057361e-07,-0.125625,0.080530,-0.007418,0.211533,0.125625,-0.092727,-0.026013,-0.322026,...,-2.660522e-06,10.406138,32.235483,-32.114244,3.486153e-16,8.750244e-14,-21124.657177,-21124.657177,-0.014149,4.731523e-08
142594,0.000104,1.069753e-07,-0.148270,0.100922,-0.025598,0.213519,0.148270,-0.117679,-0.018313,-0.329656,...,-4.552022e-06,12.943469,-18.964517,-114.034244,2.915351e-16,7.317531e-14,-527.421586,-527.421586,-0.004564,3.701998e-09


In [60]:
# The target for the train data is  before the last column
y_train = train.iloc[:,-2]
y_train

0         5
1         5
2         5
3         5
4         5
         ..
142591    9
142592    9
142593    9
142594    9
142595    9
Name: valence, Length: 142596, dtype: int64

In [61]:
# The class for the train data is the last column
pred_train = train.iloc[:,-1]
pred_train

0          neutral
1          neutral
2          neutral
3          neutral
4          neutral
            ...   
142591    positive
142592    positive
142593    positive
142594    positive
142595    positive
Name: class, Length: 142596, dtype: object

In [62]:
# The train data(undersampled) is all the columns except the last column which is the class
X_train_under = train_under.iloc[:,:-2]
X_train_under

,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,0_cc4,...,6_wl,6_mnf,6_mdf,6_pkf,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx
1425,-0.000059,-1.036992e-07,-0.054061,0.001648,-0.091893,-0.069147,0.054061,-0.002167,0.090674,0.064159,...,4.336501e-07,-22.064843,-45.689854,84.426555,-1.642886e-16,-4.123644e-14,-5044.325552,-5044.325552,0.004006,-1.450892e-08
1426,-0.000071,-1.349660e-07,-0.031998,0.023360,-0.075817,-0.041584,0.031998,-0.026572,0.064956,0.031417,...,1.534546e-06,-35.125680,-55.929854,84.426555,-1.134881e-16,-2.848552e-14,-15532.008879,-15532.008879,0.002330,2.640986e-09
1427,-0.000060,-1.205187e-07,-0.041297,0.070463,-0.072981,-0.009469,0.041297,-0.080807,0.039630,-0.006469,...,1.137351e-06,-19.255200,-35.449854,-69.173445,-1.506899e-16,-3.782317e-14,-4398.964884,-4398.964884,0.003444,9.167321e-09
1428,-0.000059,-1.144399e-07,-0.035109,0.043961,-0.058351,-0.007829,0.035109,-0.050202,0.039067,-0.008720,...,2.787986e-06,-7.458505,-14.969854,-69.173445,-1.911886e-16,-4.798835e-14,612.948582,612.948582,0.002385,1.045101e-09
1429,-0.000066,-1.108408e-07,-0.035689,0.051631,-0.070123,-0.010355,0.035689,-0.058991,0.046165,-0.008493,...,8.754477e-07,22.843478,36.230146,2.506555,-2.842338e-16,-7.134267e-14,16220.881162,16220.881162,0.001083,-6.076149e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139718,0.000256,3.292045e-07,-0.047147,0.092537,-0.035995,0.383292,0.047147,-0.105354,-0.003605,-0.605533,...,-9.839916e-07,-131.794910,-131.604517,-32.114244,5.144911e-16,1.291373e-13,-55344.039619,-55344.039619,0.010749,8.843681e-08
139720,0.000212,2.836518e-07,0.007930,0.071219,-0.061305,0.384536,-0.007930,-0.080105,0.031233,-0.637101,...,-7.840023e-07,-122.127351,-131.604517,-83.314244,1.970647e-16,4.946323e-14,-41377.312753,-41377.312753,0.016530,3.205913e-08
139722,0.000197,2.599762e-07,0.007858,0.068786,-0.076338,0.385370,-0.007858,-0.077362,0.047100,-0.651964,...,4.227782e-07,-89.989946,-90.644517,131.725756,-6.430514e-17,-1.614059e-14,-27133.350851,-27133.350851,0.013757,1.827166e-08
139724,0.000174,2.101631e-07,0.000765,0.109968,-0.160012,0.406859,-0.000765,-0.124028,0.104304,-0.737949,...,-7.793287e-07,-37.303650,-59.924517,101.005756,-1.239090e-17,-3.110115e-15,-9810.217996,-9810.217996,0.005868,-5.877859e-09


In [63]:
# The target for the train data(undersampled) is  before the last column
y_train_under = train_under.iloc[:,-2]
y_train_under

1425      8
1426      8
1427      8
1428      8
1429      8
         ..
139718    6
139720    6
139722    6
139724    6
139726    6
Name: valence, Length: 105593, dtype: int64

In [64]:
# The class for the train data(undersampled) is the last column
pred_train_under = train_under.iloc[:,-1]
pred_train_under

1425      positive
1426      positive
1427      positive
1428      positive
1429      positive
            ...   
139718     neutral
139720     neutral
139722     neutral
139724     neutral
139726     neutral
Name: class, Length: 105593, dtype: object

In [65]:
# The validation data is all the columns except the last 2 columns which are the target + class 
X_validation = validation.iloc[:,:-2]
X_validation

,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,0_cc4,...,6_wl,6_mnf,6_mdf,6_pkf,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx
0,-0.000169,-4.098298e-07,-0.060356,0.047298,0.147317,-0.008989,0.060356,-0.054022,-0.169171,0.064959,...,-2.545709e-06,-47.326305,-54.347199,-309.075322,-8.735187e-17,-2.192532e-14,10452.289503,10452.289503,0.011846,-9.987874e-09
1,-0.000141,-3.802300e-07,-0.070874,0.051835,0.222165,0.040910,0.070874,-0.059434,-0.243970,0.018670,...,-2.550875e-06,-124.319607,-218.187199,-309.075322,9.984347e-18,2.506071e-15,27763.523048,27763.523048,0.036112,3.352146e-10
2,-0.000144,-3.757562e-07,-0.084546,0.071872,0.215971,0.057898,0.084546,-0.083216,-0.244595,-0.004854,...,1.993151e-06,-93.439613,-177.227199,-309.075322,1.522556e-17,3.821615e-15,44008.348898,44008.348898,0.035138,9.668015e-09
3,-0.000169,-4.161795e-07,0.006791,0.079512,0.134118,0.029853,-0.006791,-0.089954,-0.169243,0.006249,...,2.298806e-06,-29.491657,-44.107199,-298.835322,-6.914059e-17,-1.735429e-14,44420.768513,44420.768513,0.020813,-6.116931e-09
4,-0.000182,-4.401250e-07,0.024188,0.062458,0.158687,0.047232,-0.024188,-0.070336,-0.185811,-0.013414,...,7.242016e-08,-31.927312,-33.867199,438.444678,-6.811706e-17,-1.709738e-14,27132.481327,27132.481327,0.015002,7.444352e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34017,0.000161,5.185158e-07,-0.033427,0.005838,-0.185736,-0.094345,0.033427,-0.006066,0.179824,0.059320,...,-4.579695e-06,195.095083,274.366316,195.343158,2.218619e-16,5.568733e-14,16152.988195,16152.988195,-0.035475,2.678931e-08
34018,0.000275,7.454285e-07,-0.026663,0.021879,-0.107233,-0.041092,0.026663,-0.024526,0.094874,0.009975,...,1.812504e-07,232.291748,284.606316,82.703158,1.241565e-16,3.116329e-14,26197.681460,26197.681460,-0.037842,2.182656e-08
34019,0.000346,9.383774e-07,0.137526,-0.124702,0.132393,-0.100797,-0.137526,0.136485,-0.093855,0.198813,...,5.361315e-06,243.210271,274.366316,246.543158,6.307591e-17,1.583205e-14,15459.144802,15459.144802,-0.043558,2.488220e-08
34020,0.000310,8.272760e-07,0.131100,-0.121561,0.142229,-0.090330,-0.131100,0.133433,-0.105496,0.189737,...,8.713797e-07,230.786402,274.366316,246.543158,-1.597738e-16,-4.010323e-14,36744.836234,36744.836234,-0.033643,8.543828e-09


In [66]:
# The class for the validation data is the last column
y_validation = validation.iloc[:,-2]
y_validation

0        8
1        8
2        8
3        8
4        8
        ..
34017    4
34018    4
34019    4
34020    4
34021    4
Name: valence, Length: 34022, dtype: int64

In [67]:
# The class for the validation data is the last column
pred_validation = validation.iloc[:,-1]
pred_validation

0        positive
1        positive
2        positive
3        positive
4        positive
           ...   
34017     neutral
34018     neutral
34019     neutral
34020     neutral
34021     neutral
Name: class, Length: 34022, dtype: object

In [68]:
# The test data is all the columns except the last 2 columns which are the target + class 
X_test = test.iloc[:,:-2]
X_test

,0_iav,0_aac,0_ar1,0_ar2,0_ar3,0_ar4,0_cc1,0_cc2,0_cc3,0_cc4,...,6_wl,6_mnf,6_mdf,6_pkf,6_mnp,6_ttp,6_vcf,6_psr,6_ohm,6_maxx
0,-0.000037,3.367405e-08,0.001367,-0.086367,-0.116219,-0.226394,-0.001367,0.099223,0.162996,0.303384,...,2.898422e-06,113.224527,147.831579,15.351579,-1.008898e-16,-2.532334e-14,6392.686888,6392.686888,-0.022298,-9.077232e-09
1,-0.000049,-1.305943e-08,0.020567,-0.091839,-0.112770,-0.198270,-0.020567,0.104707,0.161792,0.254321,...,-6.899484e-07,134.373149,168.311579,15.351579,-1.258649e-16,-3.159208e-14,18021.302083,18021.302083,-0.022005,-5.601545e-09
2,-0.000085,-7.680120e-08,0.039382,-0.074769,-0.087634,-0.220493,-0.039382,0.084808,0.125708,0.302672,...,2.505873e-06,126.395475,158.071579,15.351579,-1.343167e-16,-3.371349e-14,-1806.291597,-1806.291597,-0.027671,-1.806795e-08
3,-0.000088,-1.064329e-07,0.000229,-0.021488,-0.100027,-0.168621,-0.000229,0.025195,0.112178,0.218988,...,2.172300e-06,154.837829,199.031579,25.591579,-1.217489e-16,-3.055896e-14,-6582.861480,-6582.861480,-0.034137,-1.273636e-08
4,-0.000097,-1.259941e-07,0.029450,-0.046020,-0.079286,-0.161504,-0.029450,0.052715,0.103023,0.210206,...,1.913080e-06,142.995455,168.311579,25.591579,-1.561588e-16,-3.919586e-14,-2012.627914,-2012.627914,-0.030596,-1.440579e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30786,-0.000062,-1.355511e-07,0.063291,-0.048189,0.013044,-0.090411,-0.063291,0.052234,0.008427,0.145268,...,3.305718e-07,-17.014195,-19.789872,-294.815009,-5.362504e-17,-1.345989e-14,9725.693389,9725.693389,0.006284,-1.814013e-08
30787,-0.000052,-1.256089e-07,0.086627,-0.005964,-0.011104,-0.038151,-0.086627,0.005174,0.015375,0.057111,...,3.031704e-06,-31.176277,-19.789872,-294.815009,-2.841139e-17,-7.131259e-15,14121.385317,14121.385317,0.010708,-2.233756e-08
30788,-0.000025,-6.995057e-08,0.090014,0.001493,-0.082924,-0.018126,-0.090014,-0.003018,0.087261,0.003610,...,-3.159924e-07,-82.882242,-111.949872,-284.575009,-2.071889e-18,-5.200440e-16,697.097461,697.097461,0.016454,-3.479612e-09
30789,-0.000038,-9.252400e-08,0.074360,-0.010191,-0.077144,-0.019843,-0.074360,0.010122,0.087571,0.006566,...,-1.974122e-06,-83.108477,-111.949872,-130.975009,6.676131e-18,1.675709e-15,17279.908426,17279.908426,0.023264,3.795017e-09


In [69]:
# The target for the test data is  before the last column
y_test = test.iloc[:,-2]
y_test

0        4
1        4
2        4
3        4
4        4
        ..
30786    5
30787    5
30788    5
30789    5
30790    5
Name: valence, Length: 30791, dtype: int64

In [70]:
# The class for the test data is the last column
pred_test = test.iloc[:,-1]
pred_test

0        neutral
1        neutral
2        neutral
3        neutral
4        neutral
          ...   
30786    neutral
30787    neutral
30788    neutral
30789    neutral
30790    neutral
Name: class, Length: 30791, dtype: object

In [71]:
# Encoding the classes 
encoder = LabelEncoder()
pred_train = encoder.fit_transform(pred_train)
pred_train_under = encoder.transform(pred_train_under)
pred_validation = encoder.transform(pred_validation)
pred_test = encoder.transform(pred_test)

## 5. Functions and other stuff for working with ML models

In [45]:
# Defining a dictionary which will be used for mapping the results of the regression
dict = {1:0, 2:0, 3:0, 4:1, 5:1, 6:1, 7:2, 8:2, 9:2}

In [46]:
def con_matrix(y_true, pred):
    cf = confusion_matrix(y_true, pred)
    sns.heatmap(cf/np.sum(cf), annot=True, 
            fmt='.2%', cmap='Blues')

In [47]:
# Scoring the quasi classification of the model
def score_classification(y_true, pred):
    pred = np.array(list(map(lambda x: dict[x], pred)))
    f1_macro = f1_score(y_true, pred,average='macro')                       
    print(classification_report(y_true, pred))
    print("--------------------")
    print(f'The accuracy of the model is: {accuracy_score(y_true, pred)}')
    print("--------------------")
    print(f'The f1-macro score for the model is: {f1_macro}')
    print("--------------------")
    con_matrix(y_true,pred)

In [108]:
# Scoring the regression model
def score_regression(y_validation, pred):
    print(f'Mean absolute error:  {mean_absolute_error(y_validation, pred)}')
    print("--------------------")
    print(f'Mean squared error: {mean_squared_error(y_validation, pred)}')
    print("--------------------")
    print(f'Root mean squared error: {np.sqrt(mean_squared_error(y_validation, pred))}')
    print("--------------------")
    print(f'R2 score: {r2_score(y_validation,pred)}')
    print("--------------------")

In [49]:
# Score the pure classification model
def score_pure_classification(y_true, pred):
    f1_macro = f1_score(y_true, pred,average='macro')                       
    print(classification_report(y_true, pred))
    print("--------------------")
    print(f'The accuracy of the model is: {accuracy_score(y_true, pred)}')
    print("--------------------")
    print(f'The f1-macro score for the model is: {f1_macro}')
    print("--------------------")
    con_matrix(y_true,pred)

In [50]:
# Transforming the regression data
def transformData(pred):
    pred = np.floor(pred)
    pred[pred > 9] = 9
    pred[pred <= 0] = 1
    return pred

## 5.1 Classification with 3 classes

### 5.1.1 Random Forest Model

<b> Non-changed data

In [72]:
rfc1 = rfCMC(n_estimators=500, n_bins = 256, split_criterion= 1)
rfc1.fit(X_train, pred_train)

RandomForestClassifier()

In [73]:
pred = rfc1.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.47      0.35      0.40      5552
           1       0.77      0.77      0.77     23230
           2       0.33      0.42      0.37      5240

    accuracy                           0.65     34022
   macro avg       0.53      0.51      0.51     34022
weighted avg       0.66      0.65      0.65     34022

--------------------
The accuracy of the model is: 0.6496678619716654
--------------------
The f1-macro score for the model is: 0.5142354483591802
--------------------


In [74]:
pred = rfc1.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.20      0.15      0.17      7117
           1       0.68      0.71      0.69     19330
           2       0.49      0.58      0.53      4344

    accuracy                           0.56     30791
   macro avg       0.45      0.48      0.46     30791
weighted avg       0.54      0.56      0.55     30791

--------------------
The accuracy of the model is: 0.56370367964665
--------------------
The f1-macro score for the model is: 0.46448124268815133
--------------------


<b> Undersampled data

In [75]:
rfc2 = rfCMC(n_estimators=500, n_bins = 256, split_criterion= 1)
rfc2.fit(X_train_under, pred_train_under)

RandomForestClassifier()

In [76]:
pred = rfc2.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.37      0.52      0.44      5552
           1       0.82      0.64      0.72     23230
           2       0.33      0.51      0.41      5240

    accuracy                           0.60     34022
   macro avg       0.51      0.56      0.52     34022
weighted avg       0.67      0.60      0.62     34022

--------------------
The accuracy of the model is: 0.6021397918993593
--------------------
The f1-macro score for the model is: 0.5199743734920369
--------------------


In [77]:
pred = rfc2.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.24      0.27      0.25      7117
           1       0.69      0.59      0.63     19330
           2       0.44      0.63      0.52      4344

    accuracy                           0.52     30791
   macro avg       0.46      0.50      0.47     30791
weighted avg       0.55      0.52      0.53     30791

--------------------
The accuracy of the model is: 0.5196648371277321
--------------------
The f1-macro score for the model is: 0.46806504001093124
--------------------


### 5.1.2 XGBoost Model

<b> Non-changed data

In [79]:
xgbc1 = XGBClassifier(tree_method = 'gpu_hist', n_estimators = 500)
xgbc1.fit(X_train, pred_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [80]:
pred = xgbc1.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.33      0.41      0.36      5552
           1       0.77      0.71      0.74     23230
           2       0.32      0.34      0.33      5240

    accuracy                           0.60     34022
   macro avg       0.47      0.49      0.48     34022
weighted avg       0.63      0.60      0.61     34022

--------------------
The accuracy of the model is: 0.6037857856680972
--------------------
The f1-macro score for the model is: 0.4776581520553544
--------------------


In [81]:
pred = xgbc1.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.22      0.24      0.23      7117
           1       0.63      0.58      0.61     19330
           2       0.40      0.47      0.43      4344

    accuracy                           0.49     30791
   macro avg       0.42      0.43      0.42     30791
weighted avg       0.50      0.49      0.50     30791

--------------------
The accuracy of the model is: 0.48812964827384625
--------------------
The f1-macro score for the model is: 0.42226346892640665
--------------------


<b>Undersampled data

In [82]:
xgbc2 = XGBClassifier(tree_method = 'gpu_hist', n_estimators = 500)
xgbc2.fit(X_train_under, pred_train_under)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [83]:
pred = xgbc2.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.28      0.44      0.34      5552
           1       0.78      0.61      0.68     23230
           2       0.31      0.40      0.35      5240

    accuracy                           0.55     34022
   macro avg       0.45      0.48      0.46     34022
weighted avg       0.62      0.55      0.58     34022

--------------------
The accuracy of the model is: 0.5524660513785198
--------------------
The f1-macro score for the model is: 0.4577602502021956
--------------------


In [84]:
pred = xgbc2.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.22      0.33      0.26      7117
           1       0.62      0.48      0.54     19330
           2       0.36      0.46      0.41      4344

    accuracy                           0.44     30791
   macro avg       0.40      0.42      0.40     30791
weighted avg       0.49      0.44      0.46     30791

--------------------
The accuracy of the model is: 0.4388620051313696
--------------------
The f1-macro score for the model is: 0.4027619881653594
--------------------


### 5.1.3 LGMB Model

<b> Non-changed data

In [85]:
lgbmc1 = lgb.LGBMClassifier(objective="multiclass",device='gpu',n_estimators=16,learning_rate=0.07,
                           num_leaves=2591,max_depth=10,min_data_in_leaf=6140,
                           lambda_l1=16,lambda_l2=17,
                           min_gain_to_split=6.09,bagging_fraction=0.9,bagging_freq=1,feature_fraction=0.2)
lgbmc1.fit(X_train, pred_train)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l1 is set=16, reg_alpha=0.0 will be ignored. Current value: lambda_l1=16
[LightGBM] [Warning] min_gain_to_split is set=6.09, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.09
[LightGBM] [Warning] min_data_in_leaf is set=6140, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6140
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=17, reg_lambda=0.0 will be ignored. Current value: lambda_l2=17


LGBMClassifier(bagging_fraction=0.9, bagging_freq=1, device='gpu',
               feature_fraction=0.2, lambda_l1=16, lambda_l2=17,
               learning_rate=0.07, max_depth=10, min_data_in_leaf=6140,
               min_gain_to_split=6.09, n_estimators=16, num_leaves=2591,
               objective='multiclass')

In [86]:
pred = lgbmc1.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.58      0.34      0.43      5552
           1       0.78      0.84      0.81     23230
           2       0.37      0.41      0.39      5240

    accuracy                           0.69     34022
   macro avg       0.58      0.53      0.54     34022
weighted avg       0.69      0.69      0.68     34022

--------------------
The accuracy of the model is: 0.6926988419258127
--------------------
The f1-macro score for the model is: 0.5435406096044925
--------------------


In [87]:
pred = lgbmc1.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.26      0.13      0.17      7117
           1       0.69      0.81      0.75     19330
           2       0.52      0.58      0.55      4344

    accuracy                           0.62     30791
   macro avg       0.49      0.50      0.49     30791
weighted avg       0.57      0.62      0.59     30791

--------------------
The accuracy of the model is: 0.6187847098178039
--------------------
The f1-macro score for the model is: 0.4898067961189638
--------------------


<b> Undersampled data

In [88]:
lgbmc2 = lgb.LGBMClassifier(objective="multiclass",device='gpu',n_estimators=16,learning_rate=0.07,
                           num_leaves=2591,max_depth=10,min_data_in_leaf=6140,
                           lambda_l1=16,lambda_l2=17,
                           min_gain_to_split=6.09,bagging_fraction=0.9,bagging_freq=1,feature_fraction=0.2)
lgbmc2.fit(X_train_under, pred_train_under)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l1 is set=16, reg_alpha=0.0 will be ignored. Current value: lambda_l1=16
[LightGBM] [Warning] min_gain_to_split is set=6.09, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.09
[LightGBM] [Warning] min_data_in_leaf is set=6140, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6140
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=17, reg_lambda=0.0 will be ignored. Current value: lambda_l2=17


LGBMClassifier(bagging_fraction=0.9, bagging_freq=1, device='gpu',
               feature_fraction=0.2, lambda_l1=16, lambda_l2=17,
               learning_rate=0.07, max_depth=10, min_data_in_leaf=6140,
               min_gain_to_split=6.09, n_estimators=16, num_leaves=2591,
               objective='multiclass')

In [89]:
pred = lgbmc2.predict(X_validation)
score_pure_classification(pred_validation,pred)

              precision    recall  f1-score   support

           0       0.26      0.62      0.37      5552
           1       0.83      0.40      0.54     23230
           2       0.34      0.60      0.43      5240

    accuracy                           0.47     34022
   macro avg       0.48      0.54      0.45     34022
weighted avg       0.66      0.47      0.50     34022

--------------------
The accuracy of the model is: 0.4701369701957557
--------------------
The f1-macro score for the model is: 0.44661866847965204
--------------------


In [90]:
pred = lgbmc2.predict(X_test)
score_pure_classification(pred_test,pred)

              precision    recall  f1-score   support

           0       0.28      0.41      0.33      7117
           1       0.73      0.50      0.59     19330
           2       0.41      0.68      0.52      4344

    accuracy                           0.50     30791
   macro avg       0.47      0.53      0.48     30791
weighted avg       0.58      0.50      0.52     30791

--------------------
The accuracy of the model is: 0.5030366016043649
--------------------
The f1-macro score for the model is: 0.4799437656640027
--------------------


### 5.1.4 K-Nearest Neighbors Model

<b> Non-changed data

In [92]:
knnc1 = knnCMC(n_neighbors = 370)
knnc1.fit(X_train, pred_train)

KNeighborsClassifier()

In [93]:
pred = knnc1.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.38      0.23      0.28      5552
           1       0.73      0.85      0.79     23230
           2       0.40      0.29      0.34      5240

    accuracy                           0.66     34022
   macro avg       0.50      0.46      0.47     34022
weighted avg       0.62      0.66      0.63     34022

--------------------
The accuracy of the model is: 0.6598965375345365
--------------------
The f1-macro score for the model is: 0.4689458637785187
--------------------


In [94]:
pred = knnc1.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.50      0.11      0.18      7117
           1       0.69      0.83      0.76     19330
           2       0.30      0.41      0.35      4344

    accuracy                           0.61     30791
   macro avg       0.50      0.45      0.43     30791
weighted avg       0.59      0.61      0.57     30791

--------------------
The accuracy of the model is: 0.6053716995225877
--------------------
The f1-macro score for the model is: 0.4279277850631893
--------------------


<b> Undersampled data

In [96]:
# Check if 320 or 370 is better because data is undersampled
knnc2 = knnCMC(n_neighbors = 370)
knnc2.fit(X_train_under, pred_train_under)

KNeighborsClassifier()

In [97]:
pred = knnc2.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.26      0.59      0.36      5552
           1       0.78      0.54      0.63     23230
           2       0.33      0.34      0.33      5240

    accuracy                           0.51     34022
   macro avg       0.45      0.49      0.44     34022
weighted avg       0.62      0.51      0.54     34022

--------------------
The accuracy of the model is: 0.5142848744929751
--------------------
The f1-macro score for the model is: 0.44272954742016446
--------------------


In [98]:
pred = knnc2.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.31      0.27      0.28      7117
           1       0.71      0.61      0.65     19330
           2       0.28      0.51      0.36      4344

    accuracy                           0.51     30791
   macro avg       0.43      0.46      0.43     30791
weighted avg       0.55      0.51      0.53     30791

--------------------
The accuracy of the model is: 0.5144360365041732
--------------------
The f1-macro score for the model is: 0.43154539257800034
--------------------


## 5.2 Regression + Classes

### 5.2.1 Random Forest Model

<b> Non-changed data

In [102]:
rfr1 = rfCMR(n_estimators=1000, n_bins = 256, split_criterion= 6)
rfr1.fit(X_train,y_train)

RandomForestRegressor()

In [109]:
pred = rfr1.predict(X_validation)
score_regression(y_validation, pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.3591029255386582
--------------------
Mean squared error: 3.19757810092261
--------------------
Root mean squared error: 1.788177312495215
--------------------
R2 score: 0.04645222802119631
--------------------
              precision    recall  f1-score   support

           0       0.31      0.26      0.28      5552
           1       0.70      0.86      0.77     23230
           2       0.65      0.09      0.16      5240

    accuracy                           0.64     34022
   macro avg       0.55      0.40      0.41     34022
weighted avg       0.63      0.64      0.60     34022

--------------------
The accuracy of the model is: 0.6442595967315267
--------------------
The f1-macro score for the model is: 0.4054862260880412
--------------------


In [110]:
pred = rfr1.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.3434633723167497
--------------------
Mean squared error: 2.946246243616804
--------------------
Root mean squared error: 1.7164632951557117
--------------------
R2 score: 0.0008154826301808304
--------------------
              precision    recall  f1-score   support

           0       0.23      0.17      0.20      7117
           1       0.62      0.77      0.69     19330
           2       0.66      0.22      0.32      4344

    accuracy                           0.55     30791
   macro avg       0.50      0.39      0.40     30791
weighted avg       0.54      0.55      0.52     30791

--------------------
The accuracy of the model is: 0.5541879120522231
--------------------
The f1-macro score for the model is: 0.40363002075391874
--------------------


<b> Undersampled data

In [111]:
rfr2 = rfCMR(n_estimators=1000, n_bins = 256, split_criterion= 6)
rfr2.fit(X_train_under, y_train_under)

RandomForestRegressor()

In [112]:
pred = rfr2.predict(X_validation)
score_regression(y_validation, pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.3856726834528557
--------------------
Mean squared error: 3.233169930191583
--------------------
Root mean squared error: 1.798101757463015
--------------------
R2 score: 0.035838410804257426
--------------------
              precision    recall  f1-score   support

           0       0.28      0.31      0.29      5552
           1       0.70      0.82      0.75     23230
           2       0.71      0.12      0.20      5240

    accuracy                           0.63     34022
   macro avg       0.56      0.41      0.42     34022
weighted avg       0.63      0.63      0.59     34022

--------------------
The accuracy of the model is: 0.62615366527541
--------------------
The f1-macro score for the model is: 0.41668823304877356
--------------------


In [113]:
pred = rfr2.predict(X_test)
score_regression(y_test,pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.3625642229346915
--------------------
Mean squared error: 2.9654758794874927
--------------------
Root mean squared error: 1.7220557132356353
--------------------
R2 score: -0.00570602061425407
--------------------
              precision    recall  f1-score   support

           0       0.19      0.18      0.18      7117
           1       0.60      0.70      0.64     19330
           2       0.66      0.20      0.30      4344

    accuracy                           0.51     30791
   macro avg       0.48      0.36      0.38     30791
weighted avg       0.51      0.51      0.49     30791

--------------------
The accuracy of the model is: 0.5095320061056803
--------------------
The f1-macro score for the model is: 0.3778678304548058
--------------------


### 5.2.2 XGBoost Model

<b> Non-changed data

In [114]:
xgbr1 = XGBRegressor(tree_method = 'gpu_hist', n_estimators = 500)
xgbr1.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [115]:
pred = xgbr1.predict(X_validation)
score_regression(y_validation, pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.511920219062042
--------------------
Mean squared error: 3.8172130335003285
--------------------
Root mean squared error: 1.953768930426607
--------------------
R2 score: -0.13832871891775067
--------------------
              precision    recall  f1-score   support

           0       0.27      0.34      0.30      5552
           1       0.70      0.76      0.73     23230
           2       0.39      0.15      0.22      5240

    accuracy                           0.60     34022
   macro avg       0.46      0.42      0.42     34022
weighted avg       0.59      0.60      0.58     34022

--------------------
The accuracy of the model is: 0.5971430251014049
--------------------
The f1-macro score for the model is: 0.4172147919397245
--------------------


In [116]:
pred = xgbr1.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.4701108350059633
--------------------
Mean squared error: 3.495910591930468
--------------------
Root mean squared error: 1.8697354336724936
--------------------
R2 score: -0.18559667072430863
--------------------
              precision    recall  f1-score   support

           0       0.25      0.23      0.24      7117
           1       0.62      0.73      0.67     19330
           2       0.55      0.17      0.26      4344

    accuracy                           0.54     30791
   macro avg       0.47      0.38      0.39     30791
weighted avg       0.52      0.54      0.51     30791

--------------------
The accuracy of the model is: 0.5355461011334481
--------------------
The f1-macro score for the model is: 0.39019909537005604
--------------------


<b> Undersample data

In [117]:
xgbr2 = XGBRegressor(tree_method = 'gpu_hist', n_estimators = 500)
xgbr2.fit(X_train_under, y_train_under)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=0, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [118]:
pred = xgbr2.predict(X_validation)
score_regression(y_validation, pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.5209321311519692
--------------------
Mean squared error: 3.865234380914982
--------------------
Root mean squared error: 1.9660199340075324
--------------------
R2 score: -0.15264913499185728
--------------------
              precision    recall  f1-score   support

           0       0.26      0.43      0.32      5552
           1       0.70      0.69      0.70     23230
           2       0.43      0.15      0.23      5240

    accuracy                           0.57     34022
   macro avg       0.46      0.43      0.42     34022
weighted avg       0.59      0.57      0.56     34022

--------------------
The accuracy of the model is: 0.5673387807889013
--------------------
The f1-macro score for the model is: 0.41568232544713624
--------------------


In [119]:
pred = xgbr2.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.4602159235565604
--------------------
Mean squared error: 3.327249804613627
--------------------
Root mean squared error: 1.8240750545450775
--------------------
R2 score: -0.12839736237066868
--------------------
              precision    recall  f1-score   support

           0       0.24      0.28      0.26      7117
           1       0.61      0.66      0.63     19330
           2       0.57      0.19      0.28      4344

    accuracy                           0.51     30791
   macro avg       0.47      0.38      0.39     30791
weighted avg       0.52      0.51      0.50     30791

--------------------
The accuracy of the model is: 0.5059920106524634
--------------------
The f1-macro score for the model is: 0.39153849449721684
--------------------


### 5.2.3 LGBM Model

<b> Non-changed data

In [120]:
lgbmr1 = lgb.LGBMRegressor(n_estimators=18,learning_rate=0.18,num_leaves= 2420,max_depth= 3,min_data_in_leaf= 1600,lambda_l1= 0.27,lambda_l2= 62,
min_gain_to_split= 7.02,bagging_fraction= 0.5,bagging_freq=1,feature_fraction= 0.3,device='gpu',objective='regression',metric='l2')
lgbmr1.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l1 is set=0.27, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.27
[LightGBM] [Warning] min_gain_to_split is set=7.02, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.02
[LightGBM] [Warning] min_data_in_leaf is set=1600, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1600
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=62, reg_lambda=0.0 will be ignored. Current value: lambda_l2=62


LGBMRegressor(bagging_fraction=0.5, bagging_freq=1, device='gpu',
              feature_fraction=0.3, lambda_l1=0.27, lambda_l2=62,
              learning_rate=0.18, max_depth=3, metric='l2',
              min_data_in_leaf=1600, min_gain_to_split=7.02, n_estimators=18,
              num_leaves=2420, objective='regression')

In [121]:
pred = lgbmr1.predict(X_validation)
score_regression(y_validation, pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.2721221216527505
--------------------
Mean squared error: 2.8489719301589735
--------------------
Root mean squared error: 1.6878897861409594
--------------------
R2 score: 0.15040985686967245
--------------------
              precision    recall  f1-score   support

           0       0.59      0.37      0.45      5552
           1       0.73      0.94      0.82     23230
           2       0.74      0.07      0.13      5240

    accuracy                           0.71     34022
   macro avg       0.68      0.46      0.47     34022
weighted avg       0.71      0.71      0.65     34022

--------------------
The accuracy of the model is: 0.7119804832167421
--------------------
The f1-macro score for the model is: 0.4682704240057623
--------------------


In [122]:
pred = lgbmr1.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.3056902231131946
--------------------
Mean squared error: 2.913557540816502
--------------------
Root mean squared error: 1.7069146261065613
--------------------
R2 score: 0.011901469010894616
--------------------
              precision    recall  f1-score   support

           0       0.21      0.07      0.10      7117
           1       0.63      0.89      0.73     19330
           2       0.59      0.15      0.24      4344

    accuracy                           0.59     30791
   macro avg       0.48      0.37      0.36     30791
weighted avg       0.53      0.59      0.52     30791

--------------------
The accuracy of the model is: 0.5941021727128057
--------------------
The f1-macro score for the model is: 0.35908496224398073
--------------------


<b> Undersampled data

In [123]:
lgbmr2 = lgb.LGBMRegressor(n_estimators=18,learning_rate=0.18,num_leaves= 2420,max_depth= 3,min_data_in_leaf= 1600,lambda_l1= 0.27,lambda_l2= 62,
min_gain_to_split= 7.02,bagging_fraction= 0.5,bagging_freq=1,feature_fraction= 0.3,device='gpu',objective='regression',metric='l2')
lgbmr2.fit(X_train_under, y_train_under)

[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l1 is set=0.27, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.27
[LightGBM] [Warning] min_gain_to_split is set=7.02, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.02
[LightGBM] [Warning] min_data_in_leaf is set=1600, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1600
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=62, reg_lambda=0.0 will be ignored. Current value: lambda_l2=62


LGBMRegressor(bagging_fraction=0.5, bagging_freq=1, device='gpu',
              feature_fraction=0.3, lambda_l1=0.27, lambda_l2=62,
              learning_rate=0.18, max_depth=3, metric='l2',
              min_data_in_leaf=1600, min_gain_to_split=7.02, n_estimators=18,
              num_leaves=2420, objective='regression')

In [124]:
pred = lgbmr2.predict(X_validation)
score_regression(y_validation, pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.3296923049748492
--------------------
Mean squared error: 2.9670145321280788
--------------------
Root mean squared error: 1.722502404099361
--------------------
R2 score: 0.11520844612891712
--------------------
              precision    recall  f1-score   support

           0       0.41      0.45      0.43      5552
           1       0.73      0.85      0.78     23230
           2       0.72      0.11      0.18      5240

    accuracy                           0.67     34022
   macro avg       0.62      0.47      0.47     34022
weighted avg       0.67      0.67      0.63     34022

--------------------
The accuracy of the model is: 0.669155252483687
--------------------
The f1-macro score for the model is: 0.4650471962207663
--------------------


In [125]:
pred = lgbmr2.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.350320300751387
--------------------
Mean squared error: 2.9458712641340488
--------------------
Root mean squared error: 1.7163540614144999
--------------------
R2 score: 0.0009426524803968217
--------------------
              precision    recall  f1-score   support

           0       0.21      0.17      0.19      7117
           1       0.61      0.76      0.67     19330
           2       0.58      0.14      0.23      4344

    accuracy                           0.53     30791
   macro avg       0.47      0.36      0.36     30791
weighted avg       0.51      0.53      0.50     30791

--------------------
The accuracy of the model is: 0.5339222500081192
--------------------
The f1-macro score for the model is: 0.36355204334720886
--------------------


### 5.2.4 K-Nearest Neighbors Model

<b> Non-changed

In [126]:
knnr1 = knnCMR(n_neighbors = 370)
knnr1.fit(X_train, y_train)

KNeighborsRegressor()

In [127]:
pred = knnr1.predict(X_validation)
score_regression(y_validation,pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.5654282523073306
--------------------
Mean squared error: 3.8597084239609663
--------------------
Root mean squared error: 1.9646140648893273
--------------------
R2 score: -0.15100124281370153
--------------------
              precision    recall  f1-score   support

           0       0.38      0.31      0.34      5552
           1       0.71      0.89      0.79     23230
           2       0.34      0.03      0.06      5240

    accuracy                           0.66     34022
   macro avg       0.48      0.41      0.40     34022
weighted avg       0.60      0.66      0.60     34022

--------------------
The accuracy of the model is: 0.6621303862206808
--------------------
The f1-macro score for the model is: 0.3974324503183579
--------------------


In [128]:
pred = knnr1.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.4412328277743496
--------------------
Mean squared error: 3.425708811016206
--------------------
Root mean squared error: 1.8508670430412353
--------------------
R2 score: -0.16178856821648524
--------------------
              precision    recall  f1-score   support

           0       0.39      0.09      0.15      7117
           1       0.65      0.91      0.76     19330
           2       0.42      0.18      0.25      4344

    accuracy                           0.62     30791
   macro avg       0.48      0.39      0.38     30791
weighted avg       0.55      0.62      0.54     30791

--------------------
The accuracy of the model is: 0.6179078302101263
--------------------
The f1-macro score for the model is: 0.38333034210192657
--------------------


<b> Undersampled data

In [129]:
# Check if 320 or 370 is better because data is undersampled
knnr2 = knnCMR(n_neighbors = 370)
knnr2.fit(X_train_under, y_train_under)

KNeighborsRegressor()

In [130]:
pred = knnr2.predict(X_validation)
score_regression(y_validation,pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.6526953147963084
--------------------
Mean squared error: 4.173652342601846
--------------------
Root mean squared error: 2.0429518698691473
--------------------
R2 score: -0.2446222630664765
--------------------
              precision    recall  f1-score   support

           0       0.30      0.41      0.34      5552
           1       0.71      0.78      0.75     23230
           2       0.37      0.06      0.10      5240

    accuracy                           0.61     34022
   macro avg       0.46      0.42      0.40     34022
weighted avg       0.59      0.61      0.58     34022

--------------------
The accuracy of the model is: 0.6106342954558814
--------------------
The f1-macro score for the model is: 0.39624520955407916
--------------------


In [132]:
pred = knnr2.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.5044980676171609
--------------------
Mean squared error: 3.6472670585560714
--------------------
Root mean squared error: 1.9097819400539087
--------------------
R2 score: -0.23692742367266084
--------------------
              precision    recall  f1-score   support

           0       0.30      0.15      0.20      7117
           1       0.64      0.83      0.73     19330
           2       0.39      0.21      0.28      4344

    accuracy                           0.59     30791
   macro avg       0.45      0.40      0.40     30791
weighted avg       0.53      0.59      0.54     30791

--------------------
The accuracy of the model is: 0.5858530089961352
--------------------
The f1-macro score for the model is: 0.400161875846071
--------------------


## 5.3 One Vs Rest

### 5.3.1 SVC rbf OvR

<b> Non-changed data

In [133]:
svc1 = SVC(kernel = 'rbf', multiclass_strategy= 'ovr')
svc1.fit(X_train, pred_train)

SVC()

In [134]:
pred = svc1.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.39      0.13      0.20      5552
           1       0.72      0.89      0.80     23230
           2       0.43      0.28      0.34      5240

    accuracy                           0.67     34022
   macro avg       0.51      0.43      0.45     34022
weighted avg       0.62      0.67      0.63     34022

--------------------
The accuracy of the model is: 0.6745635177238257
--------------------
The f1-macro score for the model is: 0.44516593768995527
--------------------


In [135]:
pred = svc1.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.49      0.09      0.15      7117
           1       0.69      0.85      0.76     19330
           2       0.31      0.39      0.34      4344

    accuracy                           0.61     30791
   macro avg       0.49      0.44      0.42     30791
weighted avg       0.59      0.61      0.56     30791

--------------------
The accuracy of the model is: 0.6112500405962782
--------------------
The f1-macro score for the model is: 0.41799861643731323
--------------------


<b> Undersampled data

In [137]:
svc2 = SVC(kernel = 'rbf', multiclass_strategy= 'ovr')
svc2.fit(X_train_under, pred_train_under)

SVC()

In [138]:
pred = svc2.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.27      0.66      0.38      5552
           1       0.80      0.51      0.62     23230
           2       0.33      0.36      0.35      5240

    accuracy                           0.51     34022
   macro avg       0.47      0.51      0.45     34022
weighted avg       0.64      0.51      0.54     34022

--------------------
The accuracy of the model is: 0.5110810651931104
--------------------
The f1-macro score for the model is: 0.4508946552229826
--------------------


In [139]:
pred = svc2.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.29      0.30      0.30      7117
           1       0.69      0.57      0.62     19330
           2       0.28      0.49      0.35      4344

    accuracy                           0.49     30791
   macro avg       0.42      0.45      0.42     30791
weighted avg       0.54      0.49      0.51     30791

--------------------
The accuracy of the model is: 0.49352083400993796
--------------------
The f1-macro score for the model is: 0.42457749363682046
--------------------


### 5.3.2 Multiclass Logistic Regression OvR

<b> Non-changed data

In [140]:
ovr1 = OneVsRestClassifier(LogisticRegression())
ovr1.fit(X_train, pred_train)

OneVsRestClassifier()

In [141]:
pred = ovr1.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.17      0.37      0.24      5552
           1       0.63      0.29      0.40     23230
           2       0.22      0.47      0.30      5240

    accuracy                           0.33     34022
   macro avg       0.34      0.38      0.31     34022
weighted avg       0.49      0.33      0.35     34022

--------------------
The accuracy of the model is: 0.3298453941567221
--------------------
The f1-macro score for the model is: 0.3090241702154181
--------------------


In [142]:
pred = ovr1.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.09      0.04      0.06      7117
           1       0.60      0.67      0.63     19330
           2       0.29      0.41      0.34      4344

    accuracy                           0.48     30791
   macro avg       0.33      0.37      0.34     30791
weighted avg       0.44      0.48      0.46     30791

--------------------
The accuracy of the model is: 0.48432983664057677
--------------------
The f1-macro score for the model is: 0.341001262532078
--------------------


<b> Undersampled data

In [143]:
ovr2 = OneVsRestClassifier(LogisticRegression())
ovr2.fit(X_train_under, pred_train_under)

[W] [10:19:21.817544] L-BFGS: max iterations reached
[W] [10:19:21.817860] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.


OneVsRestClassifier()

In [144]:
pred = ovr2.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.20      0.54      0.29      5552
           1       0.67      0.14      0.23     23230
           2       0.22      0.61      0.33      5240

    accuracy                           0.28     34022
   macro avg       0.36      0.43      0.28     34022
weighted avg       0.52      0.28      0.26     34022

--------------------
The accuracy of the model is: 0.2777614484745165
--------------------
The f1-macro score for the model is: 0.28400602634029
--------------------


In [145]:
pred = ovr2.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.26      0.27      0.26      7117
           1       0.66      0.38      0.48     19330
           2       0.25      0.70      0.37      4344

    accuracy                           0.40     30791
   macro avg       0.39      0.45      0.37     30791
weighted avg       0.51      0.40      0.41     30791

--------------------
The accuracy of the model is: 0.39826572699814883
--------------------
The f1-macro score for the model is: 0.36962308097148217
--------------------


## 5.4 Ensemble

### 5.4.1 Classification Ensemble

In [147]:
rf = rfCMC(n_estimators=500, n_bins = 256, split_criterion= 1)
xgb = XGBClassifier(tree_method = 'gpu_hist', n_estimators = 500)
lgbm = lgb.LGBMClassifier(objective="multiclass",device='gpu',n_estimators=16,learning_rate=0.07,
                           num_leaves=2591,max_depth=10,min_data_in_leaf=6140,
                           lambda_l1=16,lambda_l2=17,
                           min_gain_to_split=6.09,bagging_fraction=0.9,bagging_freq=1,feature_fraction=0.2)

<b> Non-changed data

In [148]:
ensc1 = VotingClassifier(estimators=[('rf', rf), ('lgbm', lgbm), ('xgb', xgb)], voting='hard')
ensc1.fit(X_train, pred_train)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l1 is set=16, reg_alpha=0.0 will be ignored. Current value: lambda_l1=16
[LightGBM] [Warning] min_gain_to_split is set=6.09, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.09
[LightGBM] [Warning] min_data_in_leaf is set=6140, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6140
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=17, reg_lambda=0.0 will be ignored. Current value: lambda_l2=17


VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('lgbm',
                              LGBMClassifier(bagging_fraction=0.9,
                                             bagging_freq=1, device='gpu',
                                             feature_fraction=0.2, lambda_l1=16,
                                             lambda_l2=17, learning_rate=0.07,
                                             max_depth=10,
                                             min_data_in_leaf=6140,
                                             min_gain_to_split=6.09,
                                             n_estimators=16, num_leaves=2591,
                                             objective='multiclass')),
                             ('xgb',
                              XGBClassifier(base_score=None...
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
          

In [149]:
pred = ensc1.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.50      0.38      0.43      5552
           1       0.78      0.80      0.79     23230
           2       0.34      0.39      0.36      5240

    accuracy                           0.67     34022
   macro avg       0.54      0.52      0.53     34022
weighted avg       0.67      0.67      0.66     34022

--------------------
The accuracy of the model is: 0.6654223737581565
--------------------
The f1-macro score for the model is: 0.5271768339883884
--------------------


In [150]:
pred = ensc1.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.21      0.15      0.18      7117
           1       0.68      0.72      0.70     19330
           2       0.51      0.57      0.54      4344

    accuracy                           0.57     30791
   macro avg       0.46      0.48      0.47     30791
weighted avg       0.54      0.57      0.55     30791

--------------------
The accuracy of the model is: 0.5678607385274917
--------------------
The f1-macro score for the model is: 0.47002993758464723
--------------------


<b> Undersampled data

In [151]:
ensc2 = VotingClassifier(estimators=[('rf', rf), ('lgbm', lgbm), ('xgb', xgb)], voting='hard')
ensc2.fit(X_train_under, pred_train_under)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] lambda_l1 is set=16, reg_alpha=0.0 will be ignored. Current value: lambda_l1=16
[LightGBM] [Warning] min_gain_to_split is set=6.09, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=6.09
[LightGBM] [Warning] min_data_in_leaf is set=6140, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6140
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=17, reg_lambda=0.0 will be ignored. Current value: lambda_l2=17


VotingClassifier(estimators=[('rf', RandomForestClassifier()),
                             ('lgbm',
                              LGBMClassifier(bagging_fraction=0.9,
                                             bagging_freq=1, device='gpu',
                                             feature_fraction=0.2, lambda_l1=16,
                                             lambda_l2=17, learning_rate=0.07,
                                             max_depth=10,
                                             min_data_in_leaf=6140,
                                             min_gain_to_split=6.09,
                                             n_estimators=16, num_leaves=2591,
                                             objective='multiclass')),
                             ('xgb',
                              XGBClassifier(base_score=None...
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
          

In [152]:
pred = ensc2.predict(X_validation)
score_pure_classification(pred_validation, pred)

              precision    recall  f1-score   support

           0       0.33      0.56      0.41      5552
           1       0.82      0.58      0.68     23230
           2       0.33      0.52      0.40      5240

    accuracy                           0.57     34022
   macro avg       0.49      0.55      0.50     34022
weighted avg       0.66      0.57      0.59     34022

--------------------
The accuracy of the model is: 0.5654576450532008
--------------------
The f1-macro score for the model is: 0.4976100831351562
--------------------


In [153]:
pred = ensc2.predict(X_test)
score_pure_classification(pred_test, pred)

              precision    recall  f1-score   support

           0       0.25      0.32      0.28      7117
           1       0.69      0.55      0.61     19330
           2       0.44      0.64      0.52      4344

    accuracy                           0.51     30791
   macro avg       0.46      0.50      0.47     30791
weighted avg       0.55      0.51      0.52     30791

--------------------
The accuracy of the model is: 0.5050176999772661
--------------------
The f1-macro score for the model is: 0.4679199819422921
--------------------


### 5.4.2 Regression Ensemble

In [154]:
rf = rfCMR(n_estimators=1000, n_bins = 256, split_criterion= 6)
xgb = XGBRegressor(tree_method = 'gpu_hist', n_estimators = 500)
lgbm = lgb.LGBMRegressor(n_estimators=18,learning_rate=0.18,num_leaves= 2420,max_depth= 3,min_data_in_leaf= 1600,lambda_l1= 0.27,lambda_l2= 62,
min_gain_to_split= 7.02,bagging_fraction= 0.5,bagging_freq=1,feature_fraction= 0.3,device='gpu',objective='regression',metric='l2')

<b> Non-changed data

In [155]:
ensr1 = VotingRegressor(estimators=[('rf', rf), ('lgbm', lgbm), ('xgb', xgb)])
ensr1.fit(X_train,y_train)

[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l1 is set=0.27, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.27
[LightGBM] [Warning] min_gain_to_split is set=7.02, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.02
[LightGBM] [Warning] min_data_in_leaf is set=1600, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1600
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=62, reg_lambda=0.0 will be ignored. Current value: lambda_l2=62


VotingRegressor(estimators=[('rf', RandomForestRegressor()),
                            ('lgbm',
                             LGBMRegressor(bagging_fraction=0.5, bagging_freq=1,
                                           device='gpu', feature_fraction=0.3,
                                           lambda_l1=0.27, lambda_l2=62,
                                           learning_rate=0.18, max_depth=3,
                                           metric='l2', min_data_in_leaf=1600,
                                           min_gain_to_split=7.02,
                                           n_estimators=18, num_leaves=2420,
                                           objective='regression')),
                            ('xgb',
                             XGBRegressor(base_s...
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
    

In [156]:
pred = ensr1.predict(X_validation)
score_regression(y_validation, pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.3114566445143856
--------------------
Mean squared error: 3.0268969485209634
--------------------
Root mean squared error: 1.7397979619832193
--------------------
R2 score: 0.09735094806947442
--------------------
              precision    recall  f1-score   support

           0       0.41      0.30      0.35      5552
           1       0.71      0.90      0.80     23230
           2       0.71      0.08      0.14      5240

    accuracy                           0.68     34022
   macro avg       0.61      0.43      0.43     34022
weighted avg       0.66      0.68      0.62     34022

--------------------
The accuracy of the model is: 0.6762389042384339
--------------------
The f1-macro score for the model is: 0.42983769395256277
--------------------


In [157]:
pred = ensr1.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.3114898786889089
--------------------
Mean squared error: 2.8709915561725574
--------------------
Root mean squared error: 1.6944000578885015
--------------------
R2 score: 0.026337218539630558
--------------------
              precision    recall  f1-score   support

           0       0.23      0.14      0.17      7117
           1       0.62      0.82      0.71     19330
           2       0.69      0.19      0.29      4344

    accuracy                           0.57     30791
   macro avg       0.52      0.38      0.39     30791
weighted avg       0.54      0.57      0.53     30791

--------------------
The accuracy of the model is: 0.5721152284758534
--------------------
The f1-macro score for the model is: 0.39055228809923004
--------------------


<b> Undersampled data

In [158]:
ensr2 = VotingRegressor(estimators=[('rf', rf), ('lgbm', lgbm), ('xgb', xgb)])
ensr2.fit(X_train_under,y_train_under)

[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l1 is set=0.27, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.27
[LightGBM] [Warning] min_gain_to_split is set=7.02, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.02
[LightGBM] [Warning] min_data_in_leaf is set=1600, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1600
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=62, reg_lambda=0.0 will be ignored. Current value: lambda_l2=62


VotingRegressor(estimators=[('rf', RandomForestRegressor()),
                            ('lgbm',
                             LGBMRegressor(bagging_fraction=0.5, bagging_freq=1,
                                           device='gpu', feature_fraction=0.3,
                                           lambda_l1=0.27, lambda_l2=62,
                                           learning_rate=0.18, max_depth=3,
                                           metric='l2', min_data_in_leaf=1600,
                                           min_gain_to_split=7.02,
                                           n_estimators=18, num_leaves=2420,
                                           objective='regression')),
                            ('xgb',
                             XGBRegressor(base_s...
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
    

In [159]:
pred = ensr2.predict(X_validation)
score_regression(y_validation, pred)
score_classification(pred_validation, transformData(pred))

Mean absolute error:  1.3580586560404855
--------------------
Mean squared error: 3.120627530591843
--------------------
Root mean squared error: 1.7665297989538256
--------------------
R2 score: 0.06939960962548908
--------------------
              precision    recall  f1-score   support

           0       0.34      0.39      0.36      5552
           1       0.71      0.82      0.76     23230
           2       0.65      0.10      0.17      5240

    accuracy                           0.64     34022
   macro avg       0.57      0.44      0.43     34022
weighted avg       0.64      0.64      0.61     34022

--------------------
The accuracy of the model is: 0.6394979719005349
--------------------
The f1-macro score for the model is: 0.43255111500159527
--------------------


In [160]:
pred = ensr2.predict(X_test)
score_regression(y_test, pred)
score_classification(pred_test, transformData(pred))

Mean absolute error:  1.3272602172651313
--------------------
Mean squared error: 2.820416277643153
--------------------
Root mean squared error: 1.6794095026654914
--------------------
R2 score: 0.04348922522533494
--------------------
              precision    recall  f1-score   support

           0       0.21      0.19      0.20      7117
           1       0.61      0.74      0.66     19330
           2       0.72      0.17      0.28      4344

    accuracy                           0.53     30791
   macro avg       0.51      0.36      0.38     30791
weighted avg       0.53      0.53      0.50     30791

--------------------
The accuracy of the model is: 0.5291156506771459
--------------------
The f1-macro score for the model is: 0.3803851685972088
--------------------
